In [210]:
import json
import pandas as pd
import requests
from pandas.io.json import json_normalize
from sklearn.preprocessing import MultiLabelBinarizer

In [131]:
url = "https://www.hatvp.fr/agora/opendata/agora_repertoire_opendata.json"
json_url = requests.get(url)
json_havtp = json_url.json()

In [160]:
#with open('data.json', 'w') as outfile:
#    json.dump(json_havtp, outfile)

In [132]:
df = pd.DataFrame(json_havtp['publications'])
print(df.shape)
df.head()

(1654, 34)


,activites,adresse,affiliations,ancienNomHatvp,categorieOrganisation,clients,codePostal,collaborateurs,dateCreation,dateDernierePublicationActivite,...,nomUsage,nomUsageHatvp,pays,publierMonAdresseEmail,publierMonAdressePhysique,publierMonTelephoneDeContact,sigleHatvp,telephoneDeContact,typeIdentifiantNational,ville
0,{'listSecteursActivites': [{'label': 'Agricult...,FEDERAT DEP SYND EXPLOIT AGRICOLE YONN\nFDSEA ...,"[{'typeIdentifiantNational': 'SIREN', 'identif...",NaN,"{'label': 'Syndicat', 'categorie': 'ORGA_REP',...",[],89015,"[{'fonction': 'directeur', 'nom': 'VATINELLE',...",04/05/2018 11:30:25,06/09/2018 09:22:39,...,FDSEA de l'Yonne,FEDERATION DEPARTEMENTALE DES SYNDICATS D'EXPL...,FRANCE,True,True,True,FDSEA YONNE 89,0386494810,SIREN,AUXERRE CEDEX
1,{'listSecteursActivites': [{'label': 'Concurre...,FEDERAT FRANCAISE MAGASINS DE BRICOLAGE\n5 RUE...,"[{'typeIdentifiantNational': 'SIREN', 'identif...",NaN,"{'label': 'Organisation professionnelle', 'cat...",[],75009,"[{'fonction': 'Secrétaire Générale', 'nom': 'G...",18/04/2018 11:45:48,04/07/2018 11:43:24,...,NaN,FÉDÉRATION DES MAGASINS DE BRICOLAGE,FRANCE,True,True,False,FMB,NaN,SIREN,PARIS
2,{'listSecteursActivites': [{'label': 'Agricult...,ASS NATIONALE IND AGRO ALIMENTAIRES\r\nANIA\r\...,"[{'typeIdentifiantNational': 'SIREN', 'identif...",NaN,"{'label': 'Organisation professionnelle', 'cat...","[{'typeIdentifiantNational': 'SIREN', 'identif...",75008,[{'fonction': 'Directeur des affaires publique...,20/04/2018 14:37:14,30/04/2018 22:56:38,...,NaN,ASSOCIATION NATIONALE DES INDUSTRIES ALIMENTAIRES,FRANCE,True,True,True,ANIA,0153838600,SIREN,PARIS
3,{'listSecteursActivites': [{'label': 'Affaires...,SECOURS CATHOLIQUE\r\n106 RUE DU BAC,"[{'typeIdentifiantNational': 'RNA', 'identifia...",NaN,"{'label': 'Association', 'categorie': 'ORGA_NO...","[{'typeIdentifiantNational': 'SIREN', 'identif...",75341,[{'fonction': 'Responsable des relations insti...,30/04/2018 17:26:28,04/07/2018 11:19:48,...,NaN,NaN,FRANCE,True,True,True,NaN,0632053061,SIREN,PARIS CEDEX 07
4,{'listSecteursActivites': [{'label': 'Pouvoirs...,87 avenue Kléber,[],NaN,"{'label': 'Association', 'categorie': 'ORGA_NO...",[],75116,"[{'nom': 'PORTIER', 'prenom': 'Philippe', 'civ...",14/11/2017 11:20:57,NaN,...,NaN,NaN,FRANCE,True,True,True,AAL,0145058057,RNA,PARIS


In [133]:
def flatten_keys(dict_obj, sep="__"):
    dict_keys = list(dict_obj.keys())
    for key in dict_keys:
        if isinstance(dict_obj[key], dict):
            new_dict = flatten_keys(dict_obj[key])
            for c in new_dict:
                new_key = "{}{}{}".format(key, sep, c)
                dict_obj[new_key] = new_dict[c]
            del dict_obj[key]
    return dict_obj

flattened_dict = flatten_keys(json_havtp["publications"][0])
print(flattened_dict.keys())
nested_columns = [c for c in flattened_dict.keys() if isinstance(flattened_dict[c], list)]
print("nested_columns", nested_columns)

dict_keys(['dateDernierePublicationActivite', 'dirigeants', 'codePostal', 'identifiantNational', 'typeIdentifiantNational', 'activites__listNiveauIntervention', 'collaborateurs', 'exercices', 'declarationOrgaAppartenance', 'emailDeContact', 'affiliations', 'denomination', 'isActivitesPubliees', 'categorieOrganisation__code', 'telephoneDeContact', 'activites__listSecteursActivites', 'ville', 'declarationTiers', 'dateCreation', 'datePremierePublication', 'categorieOrganisation__label', 'nomUsageHatvp', 'lienPageFacebook', 'publierMonTelephoneDeContact', 'adresse', 'clients', 'nomUsage', 'pays', 'publierMonAdressePhysique', 'publierMonAdresseEmail', 'sigleHatvp', 'categorieOrganisation__categorie'])
nested_columns ['dirigeants', 'activites__listNiveauIntervention', 'collaborateurs', 'exercices', 'affiliations', 'activites__listSecteursActivites', 'clients']


In [134]:
data_havtp = list()
for i,ele in enumerate(json_havtp["publications"]):
    ele["id"] = i
    data_havtp.append(flatten_keys(ele))

In [135]:
list_df = []
for n_c in nested_columns:
    new_df = json_normalize(data=data_havtp, record_path=n_c, 
                           meta="id")
    list_df.append([n_c, new_df])
    
df = json_normalize(json_havtp["publications"])
list_df = [['data hatvp', df]] + list_df

In [140]:
exercice = list_df[4][1]
def my_modif(d, i):
    d["id"] = i
    return d

exercice["publicationCourante"] = exercice.apply(lambda x : my_modif(x["publicationCourante"], x["id"]), axis=1)
exercice = pd.DataFrame(list(exercice["publicationCourante"]))
list_df[4][1] = exercice

In [170]:
values = list(exercice.to_dict("index").values())
new_values = list()
for ele in values:
    if isinstance(ele["activites"], list):
        for e in ele["activites"]:
            new_ele = ele.copy()
            new_ele["activites"] = e
            new_values.append(flatten_keys(new_ele))
    else:
        new_values.append(flatten_keys(ele))

In [172]:
exercice_bis = pd.DataFrame(new_values).dropna(how="all", axis=1)
print(exercice_bis.columns)
exercice_bis.head()

Index(['activites__historique',
       'activites__publicationCourante__actionsRepresentationInteret',
       'activites__publicationCourante__domainesIntervention',
       'activites__publicationCourante__identifiantFiche',
       'activites__publicationCourante__objet',
       'activites__publicationCourante__publicationDate', 'blacklisted',
       'chiffreAffaire', 'dateDebut', 'dateFin', 'exerciceId',
       'hasNotChiffreAffaire', 'id', 'montantDepense', 'noActivite',
       'nombreActivite', 'nombreSalaries', 'publicationDate'],
      dtype='object')


,activites__historique,activites__publicationCourante__actionsRepresentationInteret,activites__publicationCourante__domainesIntervention,activites__publicationCourante__identifiantFiche,activites__publicationCourante__objet,activites__publicationCourante__publicationDate,blacklisted,chiffreAffaire,dateDebut,dateFin,exerciceId,hasNotChiffreAffaire,id,montantDepense,noActivite,nombreActivite,nombreSalaries,publicationDate
0,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,01-01-2018,31-12-2018,2553,True,0,< 10 000 euros,False,0,1.0,04/05/2018 à 11:30:57
1,[],[{'reponsablesPublics': ['Membre du Gouverneme...,[Agriculture],X2VR26I3,Présentation de l'agriculture du département d...,06/09/2018 à 09:22:39,False,NaN,01-07-2017,31-12-2017,2552,NaN,0,NaN,False,0,NaN,NaN
2,[],[{'reponsablesPublics': ['Député ; sénateur ; ...,"[Développement des territoires, Agriculture]",Z0V5SY1V,Présentation de la situation de l'élevage sur ...,06/09/2018 à 09:19:45,False,NaN,01-07-2017,31-12-2017,2552,NaN,0,NaN,False,0,NaN,NaN
3,[],[{'reponsablesPublics': ['Député ; sénateur ; ...,"[Qualité de l'eau, Agriculture]",19V0PTSH,Différents sujets abordés concernant l'agricul...,06/09/2018 à 09:21:54,False,NaN,01-07-2017,31-12-2017,2552,NaN,0,NaN,False,0,NaN,NaN
4,[],[{'reponsablesPublics': ['Membre du Gouverneme...,"[Produits chimiques, Agriculture, Accords inte...",DC34SLKH,Différents sujets abordés concernant l'agricul...,06/09/2018 à 09:20:52,False,NaN,01-07-2017,31-12-2017,2552,NaN,0,NaN,False,0,NaN,NaN


In [173]:
values = list(exercice_bis.to_dict("index").values())
new_values = list()
col = "activites__publicationCourante__actionsRepresentationInteret"
for ele in values:
    if isinstance(ele[col], list):
        for e in ele[col]:
            new_ele = ele.copy()
            new_ele[col] = e
            new_values.append(flatten_keys(new_ele))
    else:
        new_values.append(flatten_keys(ele))

In [189]:
exercice_tr = pd.DataFrame(new_values).dropna(how="all", axis=1)
print(exercice_tr.columns)
exercice_tr.head()

Index(['activites__historique',
       'activites__publicationCourante__actionsRepresentationInteret__actionMeneeAutre',
       'activites__publicationCourante__actionsRepresentationInteret__actionsMenees',
       'activites__publicationCourante__actionsRepresentationInteret__decisionsConcernees',
       'activites__publicationCourante__actionsRepresentationInteret__observation',
       'activites__publicationCourante__actionsRepresentationInteret__reponsablesPublics',
       'activites__publicationCourante__actionsRepresentationInteret__responsablePublicAutre',
       'activites__publicationCourante__actionsRepresentationInteret__tiers',
       'activites__publicationCourante__domainesIntervention',
       'activites__publicationCourante__identifiantFiche',
       'activites__publicationCourante__objet',
       'activites__publicationCourante__publicationDate', 'blacklisted',
       'chiffreAffaire', 'dateDebut', 'dateFin', 'exerciceId',
       'hasNotChiffreAffaire', 'id', 'montantDe

,activites__historique,activites__publicationCourante__actionsRepresentationInteret__actionMeneeAutre,activites__publicationCourante__actionsRepresentationInteret__actionsMenees,activites__publicationCourante__actionsRepresentationInteret__decisionsConcernees,activites__publicationCourante__actionsRepresentationInteret__observation,activites__publicationCourante__actionsRepresentationInteret__reponsablesPublics,activites__publicationCourante__actionsRepresentationInteret__responsablePublicAutre,activites__publicationCourante__actionsRepresentationInteret__tiers,activites__publicationCourante__domainesIntervention,activites__publicationCourante__identifiantFiche,...,dateDebut,dateFin,exerciceId,hasNotChiffreAffaire,id,montantDepense,noActivite,nombreActivite,nombreSalaries,publicationDate
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,01-01-2018,31-12-2018,2553,True,0,< 10 000 euros,False,0,1.0,04/05/2018 à 11:30:57
1,[],NaN,"[Inviter ou organiser des évènements, des renc...",[Actes réglementaires],NaN,[Membre du Gouvernement ou membre de cabinet m...,NaN,[FEDERAT DEP SYND EXPLOIT AGRICOLE YONNE (en p...,[Agriculture],X2VR26I3,...,01-07-2017,31-12-2017,2552,NaN,0,NaN,False,0,NaN,NaN
2,[],NaN,[Organiser des discussions informelles ou des ...,"[Lois, y compris constitutionnelles]",NaN,[Député ; sénateur ; collaborateur parlementai...,NaN,[FEDERAT DEP SYND EXPLOIT AGRICOLE YONNE (en p...,"[Développement des territoires, Agriculture]",Z0V5SY1V,...,01-07-2017,31-12-2017,2552,NaN,0,NaN,False,0,NaN,NaN
3,[],NaN,[Organiser des discussions informelles ou des ...,"[Lois, y compris constitutionnelles]",NaN,[Député ; sénateur ; collaborateur parlementai...,NaN,[FEDERAT DEP SYND EXPLOIT AGRICOLE YONNE (en p...,"[Qualité de l'eau, Agriculture]",19V0PTSH,...,01-07-2017,31-12-2017,2552,NaN,0,NaN,False,0,NaN,NaN
4,[],NaN,[Organiser des discussions informelles ou des ...,"[Lois, y compris constitutionnelles]",NaN,[Membre du Gouvernement ou membre de cabinet m...,NaN,[FEDERAT DEP SYND EXPLOIT AGRICOLE YONNE (en p...,"[Produits chimiques, Agriculture, Accords inte...",DC34SLKH,...,01-07-2017,31-12-2017,2552,NaN,0,NaN,False,0,NaN,NaN


In [218]:
exercice_qt = exercice_tr.replace("", pd.np.nan)

def empty_list_nan(l):
    if l == []:
        return pd.np.nan
    else:
        return l

for c in exercice_qt.columns:
    exercice_qt[c] = exercice_qt[c].apply(empty_list_nan)

exercice_qt.columns = exercice_qt.columns.map(lambda x : x.replace("__publicationCourante", ""))
exercice_qt.columns = exercice_qt.columns.map(lambda x : x.replace("__actionsRepresentationInteret", ""))
exercice_qt.dropna(how = "all", axis=0, inplace=True)
exercice_qt.dropna(how = "all", axis=1, inplace=True)
exercice_qt.fillna("", inplace=True)
print(exercice_qt.columns)
exercice_qt.head()

Index(['activites__actionMeneeAutre', 'activites__actionsMenees',
       'activites__decisionsConcernees', 'activites__observation',
       'activites__reponsablesPublics', 'activites__responsablePublicAutre',
       'activites__tiers', 'activites__domainesIntervention',
       'activites__identifiantFiche', 'activites__objet',
       'activites__publicationDate', 'blacklisted', 'chiffreAffaire',
       'dateDebut', 'dateFin', 'exerciceId', 'hasNotChiffreAffaire', 'id',
       'montantDepense', 'noActivite', 'nombreActivite', 'nombreSalaries',
       'publicationDate'],
      dtype='object')


,activites__actionMeneeAutre,activites__actionsMenees,activites__decisionsConcernees,activites__observation,activites__reponsablesPublics,activites__responsablePublicAutre,activites__tiers,activites__domainesIntervention,activites__identifiantFiche,activites__objet,...,dateDebut,dateFin,exerciceId,hasNotChiffreAffaire,id,montantDepense,noActivite,nombreActivite,nombreSalaries,publicationDate
0,,,,,,,,,,,...,01-01-2018,31-12-2018,2553,True,0,< 10 000 euros,False,0,1,04/05/2018 à 11:30:57
1,,"[Inviter ou organiser des évènements, des renc...",[Actes réglementaires],,[Membre du Gouvernement ou membre de cabinet m...,,[FEDERAT DEP SYND EXPLOIT AGRICOLE YONNE (en p...,[Agriculture],X2VR26I3,Présentation de l'agriculture du département d...,...,01-07-2017,31-12-2017,2552,,0,,False,0,,
2,,[Organiser des discussions informelles ou des ...,"[Lois, y compris constitutionnelles]",,[Député ; sénateur ; collaborateur parlementai...,,[FEDERAT DEP SYND EXPLOIT AGRICOLE YONNE (en p...,"[Développement des territoires, Agriculture]",Z0V5SY1V,Présentation de la situation de l'élevage sur ...,...,01-07-2017,31-12-2017,2552,,0,,False,0,,
3,,[Organiser des discussions informelles ou des ...,"[Lois, y compris constitutionnelles]",,[Député ; sénateur ; collaborateur parlementai...,,[FEDERAT DEP SYND EXPLOIT AGRICOLE YONNE (en p...,"[Qualité de l'eau, Agriculture]",19V0PTSH,Différents sujets abordés concernant l'agricul...,...,01-07-2017,31-12-2017,2552,,0,,False,0,,
4,,[Organiser des discussions informelles ou des ...,"[Lois, y compris constitutionnelles]",,[Membre du Gouvernement ou membre de cabinet m...,,[FEDERAT DEP SYND EXPLOIT AGRICOLE YONNE (en p...,"[Produits chimiques, Agriculture, Accords inte...",DC34SLKH,Différents sujets abordés concernant l'agricul...,...,01-07-2017,31-12-2017,2552,,0,,False,0,,


In [220]:
columns_label = [
    'activites__actionMeneeAutre', 'activites__actionsMenees',
    'activites__decisionsConcernees', 'activites__observation',
    'activites__reponsablesPublics', 'activites__responsablePublicAutre',
    'activites__tiers', 'activites__domainesIntervention'
]

In [227]:
mlb = MultiLabelBinarizer()
exercice_fv = exercice_qt.copy()
for c in columns_label:
    exercice_fv.drop(c, axis=1, inplace=True)
    new_df = pd.DataFrame(
            mlb.fit_transform(col_test),
            columns=mlb.classes_,
            index=col_test.index)
    new_df.columns = new_df.columns.map(lambda x : "{}__{}".format(c, x))
    exercice_fv = pd.merge(
        exercice_fv,
        new_df,
        left_index=True,
        right_index=True
    )
exercice_fv.head()

,activites__identifiantFiche,activites__objet,activites__publicationDate,blacklisted,chiffreAffaire,dateDebut,dateFin,exerciceId,hasNotChiffreAffaire,id,...,activites__domainesIntervention__Actes réglementaires,activites__domainesIntervention__Autres décisions publiques,activites__domainesIntervention__Baux emphytéotiques administratifs,activites__domainesIntervention__Contrats de cession d'un bien du domaine privé de l'État ou de ses établissements publics,activites__domainesIntervention__Contrats de concession d'une valeur supérieure aux seuils européens,activites__domainesIntervention__Contrats valant autorisation temporaire d'occupation du domaine public,activites__domainesIntervention__Décisions d'espèce,"activites__domainesIntervention__Lois, y compris constitutionnelles",activites__domainesIntervention__Marchés publics d'une valeur supérieure aux seuils européens,activites__domainesIntervention__Ordonnances de l'article 38 de la Constitution
0,,,,False,,01-01-2018,31-12-2018,2553,True,0,...,0,0,0,0,0,0,0,0,0,0
1,X2VR26I3,Présentation de l'agriculture du département d...,06/09/2018 à 09:22:39,False,,01-07-2017,31-12-2017,2552,,0,...,1,0,0,0,0,0,0,0,0,0
2,Z0V5SY1V,Présentation de la situation de l'élevage sur ...,06/09/2018 à 09:19:45,False,,01-07-2017,31-12-2017,2552,,0,...,0,0,0,0,0,0,0,1,0,0
3,19V0PTSH,Différents sujets abordés concernant l'agricul...,06/09/2018 à 09:21:54,False,,01-07-2017,31-12-2017,2552,,0,...,0,0,0,0,0,0,0,1,0,0
4,DC34SLKH,Différents sujets abordés concernant l'agricul...,06/09/2018 à 09:20:52,False,,01-07-2017,31-12-2017,2552,,0,...,0,0,0,0,0,0,0,1,0,0


In [232]:
list_df[4][1] = exercice_fv

In [233]:
writer = pd.ExcelWriter('donnees_havtp.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
for data in list_df:
    data[1].to_excel(writer, sheet_name=data[0][:31])

# Close the Pandas Excel writer and output the Excel file.
writer.save()

/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/xlsxwriter/worksheet.py:909: UserWarning: Ignoring URL 'https://www.linkedin.com/authwall?trk=ripf&trkInfo=AQHhVu8S0_I8dQAAAV1U5cZ42Rws7S-jA-x8x0E4e46QT6rZKppLzmN2c-9bl4usoDXwbbBv_PQdI12_pUWg1jfSUoMKz80y1dRz6NnQbG-GYS3Thcqsl8accYHrXRPiaOLOKVM=&originalReferer=&sessionRedirect=http%3A%2F%2Fwww.linkedin.com%2Fcompany%2F5297929' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
